<a href="https://colab.research.google.com/github/toan01-uet/FakeNewsDetec/blob/main/fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LOAD DATA**

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data_lab/pre_train_data.csv",usecols=['post_message','label'])
data['post_message'].iloc[0]

'thăng cấp_bậc hàm đối_với 2 cán_bộ chiến sỹ hy_sinh ở đà nẵng ngày 34 đại_tướng tô lâm bộ_trưởng bộ công_an đã ký quyết_định số 2398 qđ - bca-x01 thăng cấp_bậc hàm từ đại_úy lên thiếu_tá đối_với đồng_chí đặng_thanh tuấn . cùng ngày thiếu_tướng vũ xuân_viên giám_đốc công_an thành_phố đà nẵng ký quyết_định số 479 qđ - cathành phố thăng cấp_bậc hàm từ trung_sĩ lên thượng sỹ đối_với đồng_chí võ văn toàn . đây là 2 cán_bộ chiến sỹ đã hy_sinh trong quá_trình thực_hiện nhiệm_vụ đảm_bảo an_ninh trật_tự phòng_chống tội_phạm trên địa_bàn thành_phố đà nẵng . trước đó vào lúc 20 h40 ngày 242020 công_an thành_phố . đà nẵng nhận được tin báo của nhân_dân có nhóm đối_tượng đua xe và cướp_giật người đi đường tại khu_vực quận sơn_trà trong bối_cảnh toàn_quốc thực_hiện cách_ly xã_hội theo chỉ_thị số 16 của thủ_tướng chính_phủ . công_an thành_phố . đà nẵng đã chỉ_đạo cho công_an quận sơn_trà triển_khai lực_lượng truy bắt nhóm đối_tượng . trong quá_trình truy_đuổi các đối_tượng trên đại_úy đặng_thanh_tuấ

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3397 entries, 0 to 3396
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   post_message  3397 non-null   object
 1   label         3397 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 53.2+ KB


In [4]:
import re
# Import Vietnamese stop words
stopwords = set()
with open("/content/drive/MyDrive/Colab Notebooks/data_lab/vietnamese-stopwords-dash.txt", "r") as reader:
    content = reader.readlines()
    for x in content:
        stopwords.add(x.strip())
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopwords:
            words.append(word)
    return ' '.join(words)
def preprocessor(text):
    # Remove stop words
    text = remove_stopwords(text)
    # Remove emoji 
    text = deEmojify(text)

    punc_list = '.,;:?!/\|'
    for punc in punc_list:
      text = text.replace(punc, ' ')
    return text
## remove stop words
data['post_message'] = data['post_message'].apply(preprocessor)


In [5]:
data['post_message'].iloc[0]

'thăng cấp_bậc hàm 2 cán_bộ chiến sỹ hy_sinh đà nẵng 34 đại_tướng tô lâm bộ_trưởng công_an ký quyết_định 2398 qđ - bca-x01 thăng cấp_bậc hàm đại_úy thiếu_tá đồng_chí đặng_thanh tuấn   thiếu_tướng vũ xuân_viên giám_đốc công_an thành_phố đà nẵng ký quyết_định 479 qđ - cathành phố thăng cấp_bậc hàm trung_sĩ thượng sỹ đồng_chí võ văn toàn   2 cán_bộ chiến sỹ hy_sinh nhiệm_vụ an_ninh trật_tự phòng_chống tội_phạm địa_bàn thành_phố đà nẵng   20 h40 242020 công_an thành_phố   đà nẵng báo nhân_dân đối_tượng đua xe cướp_giật đi đường khu_vực sơn_trà bối_cảnh toàn_quốc cách_ly xã_hội chỉ_thị 16 thủ_tướng chính_phủ   công_an thành_phố   đà nẵng chỉ_đạo công_an sơn_trà triển_khai lực_lượng truy bắt đối_tượng   truy_đuổi đối_tượng đại_úy đặng_thanh_tuấn trung_sĩ võ văn toàn_bị tai_nạn hy_sinh   ---------- ảnh ttxviệt nam'

In [6]:
data['new_label']=['__label__'+ str(s) for s in data['label']]
data['new_data'] = [i+" "+ str(s) for (i,s) in zip(data['new_label'],data['post_message'])]
data['new_data']

0       __label__0 thăng cấp_bậc hàm 2 cán_bộ chiến sỹ...
1       __label__0 tư_vấn mùa thi nộp hồ_sơ trúng_tuyể...
2       __label__0 cơ_quan cạnh_tranh thị_trường quyết...
3       __label__0 7 ca quảng_nam hành_khách chuyến ba...
4       __label__0 học_thể dụ ‌ c thầy_giáo nguyễn_văn...
                              ...                        
3392    __label__0 liên_minh cứu sông mekong save the ...
3393    __label__0 mẹ việt nam anh_hùng 95 may khẩu_tr...
3394    __label__0 cung khan kiếm nhu_cầu áp_lực tăng_...
3395    __label__0 cảnh_tỉnh thanh_niên dân tổ     tốc...
3396    __label__0 học lái_xe hạng b2 chữa bá_bệnh lãn...
Name: new_data, Length: 3397, dtype: object

In [7]:
txt_file = "/content/drive/MyDrive/Colab Notebooks/data_lab/text_data.txt"
with open(txt_file, "w") as my_output_file:
  [ my_output_file.write(cell+'\n') for cell in data['new_data']]
  my_output_file.close()

In [8]:
!head /content/sample_data/text_data.txt

__label__0 thăng cấp_bậc hàm 2 cán_bộ chiến sỹ hy_sinh đà nẵng 34 đại_tướng tô lâm bộ_trưởng công_an ký quyết_định 2398 qđ - bca-x01 thăng cấp_bậc hàm đại_úy thiếu_tá đồng_chí đặng_thanh tuấn   thiếu_tướng vũ xuân_viên giám_đốc công_an thành_phố đà nẵng ký quyết_định 479 qđ - cathành phố thăng cấp_bậc hàm trung_sĩ thượng sỹ đồng_chí võ văn toàn   2 cán_bộ chiến sỹ hy_sinh nhiệm_vụ an_ninh trật_tự phòng_chống tội_phạm địa_bàn thành_phố đà nẵng   20 h40 242020 công_an thành_phố   đà nẵng báo nhân_dân đối_tượng đua xe cướp_giật đi đường khu_vực sơn_trà bối_cảnh toàn_quốc cách_ly xã_hội chỉ_thị 16 thủ_tướng chính_phủ   công_an thành_phố   đà nẵng chỉ_đạo công_an sơn_trà triển_khai lực_lượng truy bắt đối_tượng   truy_đuổi đối_tượng đại_úy đặng_thanh_tuấn trung_sĩ võ văn toàn_bị tai_nạn hy_sinh   ---------- ảnh ttxviệt nam
__label__0 tư_vấn mùa thi nộp hồ_sơ trúng_tuyển 15 6 6 chương_trình tư_vấn mùa thi báo thanh_niên truyền_hình đài_phát_thanh - truyền_hình bình_dương trực_tuyến fanpage fa

In [9]:
!wc /content/sample_data/text_data.txt


   3397  213021 1867606 /content/sample_data/text_data.txt


In [10]:
!head -n 2378 /content/sample_data/text_data.txt > cooking_.train
!tail -n 1019 /content/sample_data/text_data.txt > cooking_.valid

In [11]:
!pip install fasttext

In [12]:
#epoch  10,15,20,25
#ngrams 2,3
#lr e-3, e-2, e-1, 1

In [16]:
import fasttext
lrs=[0.1, 0.3, 0.5, 0.7, 0.9]
epochs=[10, 15, 20, 25]
ngrams = [2,3]
final=[0, 0, 0]
performance = 0 #f1_score
#grid search
for lr in lrs:
  for epoch in epochs:
    for ngram in ngrams:
      #train
      model = fasttext.train_supervised(input="cooking_.train", lr= lr, epoch= epoch, wordNgrams = ngram ) 
      #test
      cur_performance = model.test("cooking_.valid")[2]
      if (cur_performance > performance):
        final = [lr,epoch,ngram]
        performance = cur_performance

In [17]:
print(performance)
print(final)

0.831207065750736
[0.1, 10, 3]


In [18]:
# import fasttext
# model = fasttext.train_supervised(input='cooking_.train', autotuneValidationFile='cooking_.valid', autotuneMetric="f1:__label__1")
final_model = fasttext.train_supervised(input="cooking_.train", lr= final[0], epoch= final[1], wordNgrams = final[2] )
final_model.test("cooking_.valid")

(1019, 0.831207065750736, 0.831207065750736)

In [19]:
test_data = data.iloc[2378:]
test_data

,post_message,label,new_label,new_data
2378,15 h chiều 14-3 bình_thuận phun xịt khử_trùng ...,0,__label__0,__label__0 15 h chiều 14-3 bình_thuận phun xịt...
2379,cdc hà_nội hành_khách đi nhật sân_bay triệu_ch...,0,__label__0,__label__0 cdc hà_nội hành_khách đi nhật sân_b...
2380,bố_già mafia kinh_tế johnathan hạnh nguyễn - s...,1,__label__1,__label__1 bố_già mafia kinh_tế johnathan hạnh...
2381,vụ nữ_sinh quảng_ninh lột đồ clip tung mạng_li...,1,__label__1,__label__1 vụ nữ_sinh quảng_ninh lột đồ clip t...
2382,thạc_sĩ việt nam làm_việc bệnh_viện thâm quyến...,0,__label__0,__label__0 thạc_sĩ việt nam làm_việc bệnh_viện...
...,...,...,...,...
3392,liên_minh cứu sông mekong save the mekong kêu_...,0,__label__0,__label__0 liên_minh cứu sông mekong save the ...
3393,mẹ việt nam anh_hùng 95 may khẩu_trang chống d...,0,__label__0,__label__0 mẹ việt nam anh_hùng 95 may khẩu_tr...
3394,cung khan kiếm nhu_cầu áp_lực tăng_giá dự_án c...,0,__label__0,__label__0 cung khan kiếm nhu_cầu áp_lực tăng_...
3395,cảnh_tỉnh thanh_niên dân tổ tốc_độ bàn_thờ...,0,__label__0,__label__0 cảnh_tỉnh thanh_niên dân tổ tốc...


In [20]:
print(test_data['label'].value_counts())
print(test_data.info())

0    848
1    171
Name: label, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1019 entries, 2378 to 3396
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   post_message  1019 non-null   object
 1   label         1019 non-null   int64 
 2   new_label     1019 non-null   object
 3   new_data      1019 non-null   object
dtypes: int64(1), object(3)
memory usage: 32.0+ KB
None


In [21]:
from sklearn.metrics import confusion_matrix,recall_score,precision_score

# predict the data
test_data["predicted"] = test_data["post_message"].apply(lambda x: model.predict(x)[0][0])

# Create the confusion matrix

test_data["predict"] = test_data["predicted"].apply(lambda x: str(x)[-1])
test_data["predict"] = test_data["predict"].astype(int)

print(confusion_matrix(test_data["label"].values, test_data["predict"].values))
print(recall_score(test_data["label"].values, test_data["predict"].values))
print(precision_score(test_data["label"].values, test_data["predict"].values))


[[822  26]
 [166   5]]
0.029239766081871343
0.16129032258064516


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [22]:
test_data

,post_message,label,new_label,new_data,predicted,predict
2378,15 h chiều 14-3 bình_thuận phun xịt khử_trùng ...,0,__label__0,__label__0 15 h chiều 14-3 bình_thuận phun xịt...,__label__0,0
2379,cdc hà_nội hành_khách đi nhật sân_bay triệu_ch...,0,__label__0,__label__0 cdc hà_nội hành_khách đi nhật sân_b...,__label__0,0
2380,bố_già mafia kinh_tế johnathan hạnh nguyễn - s...,1,__label__1,__label__1 bố_già mafia kinh_tế johnathan hạnh...,__label__0,0
2381,vụ nữ_sinh quảng_ninh lột đồ clip tung mạng_li...,1,__label__1,__label__1 vụ nữ_sinh quảng_ninh lột đồ clip t...,__label__0,0
2382,thạc_sĩ việt nam làm_việc bệnh_viện thâm quyến...,0,__label__0,__label__0 thạc_sĩ việt nam làm_việc bệnh_viện...,__label__0,0
...,...,...,...,...,...,...
3392,liên_minh cứu sông mekong save the mekong kêu_...,0,__label__0,__label__0 liên_minh cứu sông mekong save the ...,__label__0,0
3393,mẹ việt nam anh_hùng 95 may khẩu_trang chống d...,0,__label__0,__label__0 mẹ việt nam anh_hùng 95 may khẩu_tr...,__label__0,0
3394,cung khan kiếm nhu_cầu áp_lực tăng_giá dự_án c...,0,__label__0,__label__0 cung khan kiếm nhu_cầu áp_lực tăng_...,__label__0,0
3395,cảnh_tỉnh thanh_niên dân tổ tốc_độ bàn_thờ...,0,__label__0,__label__0 cảnh_tỉnh thanh_niên dân tổ tốc...,__label__0,0


In [23]:
test_data['predict'].value_counts()

0    988
1     31
Name: predict, dtype: int64